In [56]:
import pandas as pd
import numpy as np
from sklearn.model_selection import TimeSeriesSplit
from datetime import datetime
import statsmodels.api as sm

stations = ['Jämeräntaival']
data = pd.read_csv('datasets/' + stations[0] + '_hourly_aggregate.csv')
data['Departure'] = pd.to_datetime(data['Departure'], format='mixed')

results = []

weather_df = pd.read_csv('datasets/weather_hourly_helsinki.csv')
weather_df = weather_df.loc[1:, :]
weather_df.columns = weather_df.iloc[0]
weather_df = weather_df.loc[2:, :]
weather_df['time'] = pd.to_datetime(weather_df['time'], format='mixed')

data = pd.merge(weather_df, data, how='inner', left_on='time', right_on='Departure')
data = data.drop(['time'], axis=1)

data['temperature_2m (°C)'] = pd.to_numeric(data['temperature_2m (°C)'], errors='coerce')
data['rain (mm)'] = pd.to_numeric(data['rain (mm)'], errors='coerce')
data['trip'] = pd.to_numeric(data['trip'], errors='coerce')


# generation of weekday & hour series
datedata = pd.DataFrame()
datedata['date'] = data['Departure']
datedata['weekday'] = data['Departure'].dt.weekday
days = ['mon', 'tue', 'wed', 'thu', 'fri', 'sat']
for day in days:
    datedata[day] = 0
    datedata.loc[datedata['date'].dt.weekday == 0, day] = 1
for i in range(0, 23):
    asd = 'hour' + str(i)
    days.append(asd)
    datedata[asd] = 0
    datedata.loc[datedata['date'].dt.hour == i, asd] = 1

data = pd.merge(datedata, data, how='inner', left_on='date', right_on='Departure')
data.set_index(data['Departure'], inplace=True)



train_end = datetime(year=2022, month=10, day=31)
train_start = datetime(year=2018,month=4,day=1)
train_data = data[:train_end]
test_end = datetime(year=2023, month=10, day = 31)
test_data  = data[datetime(year=2023,month=4,day=1):]


def MASE(y_true, y_pred, y_train):
    forecast = y_pred.reset_index(drop=True)
    outsample = y_true[:].iloc[:len(y_pred)]
    insample = y_train.reset_index(drop=True).to_numpy()
    frequency=1
    return np.mean(np.abs(forecast - outsample)) / np.mean(np.abs(insample[:-frequency] - insample[frequency:]))


# reset indexes as they do funky things
test_data.reset_index(drop=True, inplace=True)
train_data.reset_index(drop=True, inplace=True)

print('')

# no exogenous variables at all
model = sm.tsa.statespace.SARIMAX(train_data['trip'], order=(3,1,2), seasonal_order=(1, 1, 1, 24)).fit()
forecast = model.forecast(steps=24*30*7)
forecast = forecast[:]
results.append((MASE(test_data['trip'], forecast, train_data['trip']), 'no exog'))

#just the weather as exogenous data
exogenous = ['rain (mm)', 'temperature_2m (°C)']
model = sm.tsa.statespace.SARIMAX(train_data['trip'], order=(3,1,2), seasonal_order=(1, 1, 1, 24), exog=train_data[exogenous]).fit()
forecast = model.forecast(steps=24*30*7, exog=test_data[exogenous].iloc[:(24*30*7)])
forecast = forecast[:]
results.append((MASE(test_data['trip'], forecast, train_data['trip']), 'just weather'))

#just weekday + time of the day as exogenous
model = sm.tsa.statespace.SARIMAX(train_data['trip'], order=(3,1,2), seasonal_order=(1, 1, 1, 24), exog=train_data[days]).fit()
forecast = model.forecast(steps=24*30*7, exog=test_data[days].iloc[:(24*30*7)])
forecast = forecast[:]
results.append((MASE(test_data['trip'], forecast, train_data['trip']), 'just timely stuff'))


#all exogenous variables
for asd in days:
    exogenous.append(asd)
model = sm.tsa.statespace.SARIMAX(train_data['trip'], order=(3,1,2), seasonal_order=(1, 1, 1, 24), exog=train_data[exogenous]).fit()
forecast = model.forecast(steps=24*30*7, exog=test_data[exogenous].iloc[:(24*30*7)])
forecast = forecast[:]
results.append((MASE(test_data['trip'], forecast, train_data['trip']), 'all exogs'))


print(results)
    



asd
asd2


/Users/otto/.pyenv/versions/3.11.2/lib/python3.11/site-packages/statsmodels/tsa/statespace/sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
/Users/otto/.pyenv/versions/3.11.2/lib/python3.11/site-packages/statsmodels/tsa/statespace/sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            8     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  2.52687D+00    |proj g|=  3.92148D-01


 This problem is unconstrained.



At iterate    5    f=  2.24021D+00    |proj g|=  1.15085D-01

At iterate   10    f=  2.19412D+00    |proj g|=  5.75993D-03

At iterate   15    f=  2.19337D+00    |proj g|=  8.22902D-04

At iterate   20    f=  2.19327D+00    |proj g|=  4.98625D-03

At iterate   25    f=  2.19325D+00    |proj g|=  3.48694D-05

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    8     25     26      1     0     0   3.487D-05   2.193D+00
  F =   2.1932501470633761     

CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH             


/Users/otto/.pyenv/versions/3.11.2/lib/python3.11/site-packages/statsmodels/tsa/statespace/sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
/Users/otto/.pyenv/versions/3.11.2/lib/python3.11/site-packages/statsmodels/tsa/statespace/sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =           10     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  2.52689D+00    |proj g|=  3.92567D-01


 This problem is unconstrained.



At iterate    5    f=  2.23987D+00    |proj g|=  1.15352D-01

At iterate   10    f=  2.19343D+00    |proj g|=  5.82758D-03

At iterate   15    f=  2.19242D+00    |proj g|=  1.41481D-03

At iterate   20    f=  2.19228D+00    |proj g|=  1.02376D-03

At iterate   25    f=  2.19225D+00    |proj g|=  8.93884D-04

At iterate   30    f=  2.19224D+00    |proj g|=  6.02699D-05

At iterate   35    f=  2.19224D+00    |proj g|=  4.09553D-04

At iterate   40    f=  2.19223D+00    |proj g|=  4.81752D-04

At iterate   45    f=  2.19220D+00    |proj g|=  9.39974D-04

At iterate   50    f=  2.19205D+00    |proj g|=  5.16353D-03

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tn

/Users/otto/.pyenv/versions/3.11.2/lib/python3.11/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
